In [2]:
%run Reddit_import_and_functions.ipynb

pd.set_option('display.max_colwidth',350)

In [2]:
list_all_hdf5("masterdata.hdf5")

Train1Ytrain
Train1dset
Train2dset
X_Train2
columnsums
columnsums/columnindextodrop
columnsums/columnsumshalf0
columnsums/columnsumshalf1
environment
rowsums
rowsums/columnsumsCrypto
rowsums/columnsumsEnv
rowsums/columnsumsTrain2
rowsums/crypto_dset
rowsums/env_dset
rowsums/rowsumsCrypto
rowsums/rowsumsEnv
rowsums/rowsumsTrain2
rowsums/rowsumshalf0
rowsums/rowsumshalf1
y_train2


In [3]:
with h5py.File("masterdata.hdf5", "r") as z:
#     dset = z["rowsums/env_dset"]
#     X_train_env = dset[:]
#     print(X_train_env.shape)
    dset = z["rowsums/crypto_dset"]
    X_train_crypto = dset[:]
    print(X_train_crypto.shape)

(293358, 17847)


In [8]:
dfraw = pd.read_csv('Train1allinfo.csv')
dfraw.columns

Index(['Unnamed: 0', 'index', 'datetime', 'corpus', 'permalink', 'author',
       'subreddit'],
      dtype='object')

In [5]:
envmask = dfraw['subreddit'].astype(bool)
cryptomask = 1 - envmask
cryptomask = cryptomask.astype(bool)
print(cryptomask.sum())
envmask.sum()

293358


146787

In [9]:
df = dfraw[cryptomask]
dfraw = None

In [6]:
headers = pd.read_csv('masterFeaturesVector.csv')


In [10]:
X_train_crypto = pd.DataFrame(data=X_train_crypto, dtype='int16', 
                      index=df.index)
X_train_crypto.shape

(293358, 17847)

In [14]:
df['datetime'].sort_values(ascending=False) # 1.545105e+09
df['datetime'].sort_values() # 1.504570e+09
endtimestamp = 1.545105e+09
starttimestamp = 1.504570e+09

In [18]:
date_stamp0 = datetime.datetime.utcfromtimestamp(int(starttimestamp)).timestamp()


In [19]:
date_tuple = datetime.datetime.utcfromtimestamp(int(starttimestamp))
year = date_tuple.year
month = date_tuple.month
day = date_tuple.day
hour = date_tuple.hour
second = date_tuple.second
year, month, day, hour, second

(2017, 9, 5, 0, 40)

In [21]:
date_stamp1 = datetime.datetime(2017, 12, 17, 0, 40).timestamp()
date_stamp2start = datetime.datetime(2017, 12, 19, 0, 40).timestamp()
date_stamp2 = datetime.datetime(2018, 2, 8, 0, 40).timestamp()
date_stamp3 = datetime.datetime(2018, 7, 1, 0, 40).timestamp()
date_stamp4 = datetime.datetime(2018, 10, 1, 0, 40).timestamp()

In [22]:
years1mask = (df['datetime'] > date_stamp0) & (df['datetime'] < date_stamp1)
years1mask.sum()

54635

In [23]:
years2mask = (df['datetime'] > date_stamp2start) & (df['datetime'] < date_stamp2)
years2mask.sum()

71310

In [24]:
years3mask = (df['datetime'] > date_stamp3) & (df['datetime'] < date_stamp4)
years3mask.sum()

44430

In [25]:
with h5py.File("masterdata.hdf5",'a') as f :
    dset = f['columnsums/columnindextodrop']
    columnindextodrop = dset[:]

In [24]:
keepcols = set(range(17847)).symmetric_difference(columnindextodrop)
print(len(keepcols))
keepcols = list(keepcols)

10584


In [26]:
X_train_crypto.drop(columnindextodrop, axis=1, inplace=True)

In [27]:
X_train_crypto.loc[df[years1mask].index,'date_class'] = 0

In [28]:
X_train_crypto.loc[df[years2mask].index,'date_class'] = 1

In [29]:
X_train_crypto.loc[df[years3mask].index,'date_class'] = 2

In [30]:
X_train_crypto['date_class'].value_counts()

1.0    71310
0.0    54635
2.0    44430
Name: date_class, dtype: int64

In [31]:
X_train_crypto.shape

(293358, 10585)

In [32]:
X_train_crypto.dropna(axis=0, inplace=True)

In [33]:
X_train_crypto.shape

(170375, 10585)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_train_crypto.drop('date_class',axis=1), 
                                                    X_train_crypto['date_class'],train_size=0.5)

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [35]:
# all parameters will be kept at default except possible tweaking of alpha
nb = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nbmodel = nb.fit(X_train, y_train)

In [36]:
pred1 = nbmodel.predict(X_train)
predtest = nbmodel.predict(X_test)

In [37]:
results = pd.DataFrame(index=X_train_crypto.index)

In [38]:
predfinal = pd.Series(data=pred1, index=X_train.index, name='nb3_pred')

In [39]:
results['nb3_pred'] = predfinal

In [40]:
results['nb3_errors'] = results['nb3_pred'] - y_train

In [41]:
confusion_matrix(y_train, predfinal)

array([[13676,  9427,  4164],
       [ 5776, 25100,  4859],
       [ 2631,  4630, 14924]], dtype=int64)

In [42]:
confusion_matrix(y_train, predfinal) / y_train.shape[0]

array([[0.16054093, 0.11066243, 0.0488807 ],
       [0.06780377, 0.2946459 , 0.05703922],
       [0.03088499, 0.05435102, 0.17519105]])

In [43]:
confusion_matrix(y_test, predtest)

array([[12674,  9926,  4768],
       [ 6203, 24362,  5010],
       [ 2915,  4891, 14439]], dtype=int64)

In [44]:
confusion_matrix(y_test, predtest) / y_test.shape[0]

array([[0.14877682, 0.11651876, 0.05597032],
       [0.07281542, 0.28597925, 0.0588111 ],
       [0.03421843, 0.05741419, 0.1694957 ]])

In [ ]:
results.to_csv('crypto_nbdates.csv')